## Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [22]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [23]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=[]
df = pd.DataFrame(application_df["APPLICATION_TYPE"].value_counts().reset_index())
df=df.reset_index()

for x in range(len(df)):
    if df["count"][x] < 200:
        application_types_to_replace.append(df["APPLICATION_TYPE"][x])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [24]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_data = pd.DataFrame(application_df["CLASSIFICATION"].value_counts()).reset_index()
classification_data

,CLASSIFICATION,count
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C4120,1
67,C8210,1
68,C2561,1
69,C4500,1


In [25]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_data[classification_data["count"]>1]

,CLASSIFICATION,count
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
5,C7000,777
6,C1700,287
7,C4000,194
8,C5000,116
9,C1270,114


In [26]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace=[]
for x in range(len(classification_data)):
    if classification_data["count"][x] < 1800:
        classifications_to_replace.append(classification_data["CLASSIFICATION"][x])


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(application_df)

In [28]:
# Split our preprocessed data into our features and target arrays
X = df.drop("SPECIAL_CONSIDERATIONS_Y", axis = 1)
y = df["SPECIAL_CONSIDERATIONS_Y"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=1)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [30]:

nn = tf.keras.models.Sequential()

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units = 80, activation = "relu", input_dim = 43))

# Second hidden layer
nn.add(Dense(units = 30, activation = "relu", input_dim = 2))

# Output layer
nn.add(Dense(units = 1, activation = "relu", input_dim = 2))


# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# Compile the model SGD NAG Adam
nn.compile(optimizer =  "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [41]:
# Train the model
model_nn_fitted = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100


c:\Users\vallk\anaconda3\Lib\site-packages\keras\src\losses\losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 577us/step - accuracy: 0.9988 - loss: nan
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - accuracy: 0.9993 - loss: nan
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.9989 - loss: nan
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - accuracy: 0.9990 - loss: nan
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - accuracy: 0.9991 - loss: nan
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.9992 - loss: nan
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step - accuracy: 0.9990 - loss: nan
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - accuracy: 0.9989 - loss: nan
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - accuracy: 0.9991 - loss: nan
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - accuracy: 0.9993 - loss: nan
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - accuracy: 0.9993 - loss: nan
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - accura

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 701us/step - accuracy: 0.9994 - loss: nan
Loss: nan, Accuracy: 0.9994168877601624


In [43]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')